In [1]:
import subprocess
import importlib
import sys

def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
        print(f"{package_name} is already installed")
    except ImportError:
        print(f"Installing {package_name} ...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"{package_name} installation completed")
        except subprocess.CalledProcessError as e:
            print(f"{package_name} installation failed (exit code {e.returncode})")

install_package("tensorflow")           
install_package("cv2", "opencv-python") 
install_package("PIL", "Pillow")        

tensorflow is already installed
Installing opencv-python ...
opencv-python installation completed
Pillow is already installed


In [8]:
import tensorflow as tf
import numpy as np
import cv2
import h5py
from PIL import Image, ImageFont, ImageDraw

In [10]:
f = h5py.File("keras_model.h5", mode="r+")
model_config_string = f.attrs.get("model_config")
if model_config_string.find('"groups": 1,') != -1:
    model_config_string = model_config_string.replace('"groups": 1,', '')
    f.attrs.modify('model_config', model_config_string)
    f.flush()
    model_config_string = f.attrs.get("model_config")
    assert model_config_string.find('"groups": 1,') == -1
f.close()

In [12]:
import platform
import gc  # 가비지 컬렉터 모듈

# Suppress scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the Keras model and class names
model = tf.keras.models.load_model("keras_Model.h5", compile=False)
class_names = [line.strip() for line in open("labels.txt", "r", encoding='utf-8').readlines()]

# Initialize video capture
capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

if not capture.isOpened():
    print("비디오 입력에 오류가 있습니다.")
    exit()

# Define font path based on the operating system
system_name = platform.system()

if system_name == "Windows":
    font_path = "C:/Windows/Fonts/malgun.ttf"
elif system_name == "Darwin":
    font_path = "/Library/Fonts/AppleGothic.ttf"
else:
    font_path = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# Load the font once outside the loop
try:
    font = ImageFont.truetype(font_path, 20)
except IOError:
    print("폰트를 찾을 수 없습니다. 기본 폰트를 사용합니다. 한글이 제대로 표시되지 않을 수 있습니다.")
    font = ImageFont.load_default()

while True:
    ret, frame = capture.read()
    
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Exit on pressing 'x' or 'ESC'
    key = cv2.waitKey(1) & 0xFF
    if key == ord('x') or key == 27:
        break

    # Preprocess the frame for prediction
    size = (224, 224)
    img = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)
    img = (img / 127.0) - 1  # Normalize to [-1, 1]
    img = img.reshape((1, 224, 224, 3))

    # Make prediction
    prediction = model.predict(img)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Prepare text for display (한글 지원)
    label_text = f"{class_name[2:]}: {confidence_score * 100:.2f}%"  # 필요에 따라 class_name 슬라이싱 조절

    # Convert OpenCV image (BGR) to PIL image (RGB)
    cv2_im_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_im = Image.fromarray(cv2_im_rgb)

    # Initialize ImageDraw
    draw = ImageDraw.Draw(pil_im)

    # Define text position
    text_position = (10, 10)  # 왼쪽 상단 위치 조절 가능

    # Define text color (검은색)
    text_color = (0, 0, 0)  # RGB

    # Add text to the PIL image
    draw.text(text_position, label_text, font=font, fill=text_color)

    # Convert PIL image back to OpenCV image (BGR)
    cv2_im_processed = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)

    # Display the frame
    cv2.imshow("VideoFrame", cv2_im_processed)

    # PIL 이미지와 관련 객체 해제
    del pil_im
    del draw
    del cv2_im_rgb
    gc.collect()  # 가비지 컬렉터

# Release resources
capture.release()
cv2.destroyAllWindows()

C:\Users\119hw\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


비디오 입력에 오류가 있습니다.
Error: Failed to capture image.
